In [1]:
#from model.model import EncoderWithTime, Reparametrize, DecoderNoTime, Regressor, Model
from sklearn.metrics import r2_score
from preprocess import prepare_df, normalize
from tqdm import tqdm
import json
import torch
import pandas as pd
import argparse
import numpy as np
import time

In [15]:
import pandas as pd
device = torch.device("cuda:" + str(1))
data = pd.read_csv('../lstm/utils/df_imputed.csv', index_col=0).drop(columns =
                                                    ['date','TA_F_DAY','TA_F_NIGHT','P_F'])
raw = pd.read_csv('../data/df_20210510.csv', index_col=0)['GPP_NT_VUT_REF']
df_sensor, df_meta, df_gpp = prepare_df(data)

In [17]:
import operator
sites = raw.index.unique()
masks = []
for s in sites:
    mask = raw[raw.index == s].isna().values
    masks.append(list(map(operator.not_, mask)))

In [18]:
sites_to_train = list(range(len(df_sensor)))
sites_to_train.remove(0)
sites_to_test = [0]

x_train = [df_sensor[i].values for i in sites_to_train]
conditional_train = [df_meta[i].values for i in sites_to_train]
y_train = [df_gpp[i].values.reshape(-1,1) for i in sites_to_train]

x_test = [df_sensor[i].values for i in sites_to_test]
conditional_test = [df_meta[i].values for i in sites_to_test]
y_test = [df_gpp[i].values.reshape(-1,1) for i in sites_to_test]

In [21]:
import torch.nn as nn
import torch, math
import time

class Transformer(nn.Module):
    # d_model : number of features
    def __init__(self,feature_size= 8,num_layers=3,dropout=0):
        super(Transformer, self).__init__()

        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=4, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)        
        self.decoder = nn.Linear(feature_size,1)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1    
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def forward(self, src, device):
        
        mask = self._generate_square_subsequent_mask(len(src)).to(device)
        output = self.transformer_encoder(src,mask)
        output = self.decoder(output)
        return output

In [22]:

model = Transformer(feature_size=8).to(device)
optimizer = torch.optim.Adam(model.parameters())
criterion = torch.nn.MSELoss()
best_model = ""
min_train_loss = float('inf')

for epoch in range(100):
        train_loss = 0
        val_loss = 0
        train_r2 = 0
        ## TRAIN -- TEACHER FORCING
        model.train()
        for (x, y) in zip(x_train, y_train):
            optimizer.zero_grad()
            x = torch.FloatTensor(x).unsqueeze(1).to(device)
            y = torch.FloatTensor(y).to(device)
            y_pred = model(x,device).squeeze(1)
            loss = criterion( y_pred, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.detach().item()
            train_r2 += r2_score(y_true=y.detach().cpu().numpy(), y_pred=y_pred.detach().cpu().numpy())
        model.eval()
        with torch.no_grad():
              for (x, y) in zip(x_test, y_test):
                x = torch.FloatTensor(x).unsqueeze(1).to(device)
                y = torch.FloatTensor(y).to(device)
                y_pred = model(x,device).squeeze(1)
                test_loss = criterion(y_pred, y)
                test_r2 = r2_score(y_true=y.detach().cpu()[masks[0]], y_pred=y_pred.detach().cpu()[masks[0]])


        print(f"Epoch: {epoch+1} ")
        print(f"Train loss: {train_loss / len(sites_to_train):.2f} | R2: {train_r2 / len(sites_to_train):.2f}")
        print(f"Test loss: {test_loss:.2f} | R2: {test_r2:.2f}")      

Epoch: 1 
Train loss: 0.64 | R2: 0.36
Test loss: 0.93 | R2: 0.10
Epoch: 2 
Train loss: 0.58 | R2: 0.42
Test loss: 1.02 | R2: 0.01
Epoch: 3 
Train loss: 0.51 | R2: 0.49
Test loss: 0.88 | R2: 0.15
Epoch: 4 
Train loss: 0.44 | R2: 0.56
Test loss: 0.75 | R2: 0.28
Epoch: 5 
Train loss: 0.40 | R2: 0.60
Test loss: 0.71 | R2: 0.33


KeyboardInterrupt: 

In [36]:
torch.cuda.empty_cache()

In [9]:
!nvidia-smi

Wed May 12 19:45:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  TITAN X (Pascal)    Off  | 00000000:04:00.0 Off |                  N/A |
| 71%   83C    P2   234W / 250W |  11855MiB / 12196MiB |     89%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  TITAN X (Pascal)    Off  | 00000000:05:00.0 Off |                  N/A |
| 27%   

In [ ]:
|